## Building A Chatbot
How to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions


In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

In [4]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002A7CDEBB0D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002A7CE00C090>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [5]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I am Ippo Makunouchi. I am a featherweight boxing champion.")])

AIMessage(content='That\'s amazing, Ippo!  A featherweight boxing champion! \n\nI\'ve heard tales of your incredible fighting spirit and powerful "demon fist."  \n\nWhat\'s it like being at the top of your game? Is there anything you\'re looking forward to in your boxing career?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 28, 'total_tokens': 93, 'completion_time': 0.118181818, 'prompt_time': 0.002161635, 'queue_time': 0.244282205, 'total_time': 0.120343453}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--395a40c1-c212-4943-957e-d14fefd417c3-0', usage_metadata={'input_tokens': 28, 'output_tokens': 65, 'total_tokens': 93})

In [6]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, I am Ippo Makunouchi. I am a featherweight boxing champion."),
    AIMessage(content="That\'s amazing, Ippo!  A featherweight boxing champion! \n\nI\'ve heard tales of your incredible fighting spirit and powerful 'demon fist.'  \n\nWhat\'s it like being at the top of your game? Is there anything you\'re looking forward to in your boxing career?\n"),
    HumanMessage(content="What is my name & What am I?")
])

AIMessage(content='You are Ippo Makunouchi, a featherweight boxing champion! 🥊🏆 \n\nYou\'re known for your indomitable spirit and powerful "demon fist". \n\n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 110, 'total_tokens': 150, 'completion_time': 0.072727273, 'prompt_time': 0.005556319, 'queue_time': 0.245183279, 'total_time': 0.078283592}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--633facab-342e-4952-8933-47db54414595-0', usage_metadata={'input_tokens': 110, 'output_tokens': 40, 'total_tokens': 150})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [7]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [8]:
store={}

In [9]:
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [10]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [11]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x000002A7CFC2D8A0>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [12]:
config = {"configurable":{"session_id":"chat1"}}

In [14]:
response = with_message_history.invoke([
    HumanMessage(content="Hi, I am Ippo Makunouchi. I am a featherweight boxing champion."),
], config=config)

In [15]:
response

AIMessage(content="Wow, Ippo Makunouchi!  It's an honor to meet you.  \n\nThe featherweight champion! That's incredible. \n\nWhat's it like being at the top of your game? What's your training like?  Any big fights coming up? \n\nI'm a huge fan of boxing, and I'm really impressed by your dedication and skill.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 86, 'prompt_tokens': 28, 'total_tokens': 114, 'completion_time': 0.156363636, 'prompt_time': 0.002152136, 'queue_time': 0.248539633, 'total_time': 0.158515772}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--707b682f-6e4d-4108-b1c9-fa52126ba522-0', usage_metadata={'input_tokens': 28, 'output_tokens': 86, 'total_tokens': 114})

In [16]:
response.content

"Wow, Ippo Makunouchi!  It's an honor to meet you.  \n\nThe featherweight champion! That's incredible. \n\nWhat's it like being at the top of your game? What's your training like?  Any big fights coming up? \n\nI'm a huge fan of boxing, and I'm really impressed by your dedication and skill.\n"

In [17]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config
)

AIMessage(content="Your name is Ippo Makunouchi! 🥊  \n\nYou're the featherweight boxing champion, remember? 😄  \n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 127, 'total_tokens': 157, 'completion_time': 0.054545455, 'prompt_time': 0.005648659, 'queue_time': 0.246847739, 'total_time': 0.060194114}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--1915529a-705c-420a-9af1-2bad07ba4dc6-0', usage_metadata={'input_tokens': 127, 'output_tokens': 30, 'total_tokens': 157})

In [18]:
config1={"configurable":{"session_id":"chat2"}}

In [19]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to learn it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 14, 'total_tokens': 53, 'completion_time': 0.070909091, 'prompt_time': 0.001917984, 'queue_time': 0.24221005399999998, 'total_time': 0.072827075}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--b5a80ef4-0b8e-4357-9325-167db1845ded-0', usage_metadata={'input_tokens': 14, 'output_tokens': 39, 'total_tokens': 53})

In [20]:
with_message_history.invoke(
    [HumanMessage(content="Hey, my name is John?")],
    config=config1
)

AIMessage(content="Hi John! It's nice to meet you. 👋\n\nIs there anything I can help you with today? 😄  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 68, 'total_tokens': 96, 'completion_time': 0.050909091, 'prompt_time': 0.003817109, 'queue_time': 0.24392626099999998, 'total_time': 0.0547262}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--444e9e95-b12e-44d5-bdac-0ed060805cc2-0', usage_metadata={'input_tokens': 68, 'output_tokens': 28, 'total_tokens': 96})

In [21]:
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)

AIMessage(content="You told me your name is John! 😊  \n\nIs there anything else you'd like to talk about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 109, 'total_tokens': 135, 'completion_time': 0.047272727, 'prompt_time': 0.005673564, 'queue_time': 0.24108505500000002, 'total_time': 0.052946291}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--46b6ca84-8457-45c1-9d2e-ae5e4f87c360-0', usage_metadata={'input_tokens': 109, 'output_tokens': 26, 'total_tokens': 135})

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [22]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

In [23]:
prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
    MessagesPlaceholder(variable_name="messages")
])

In [24]:
chain = prompt|model

In [25]:
chain

ChatPromptTemplate(input_variables=['messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langchain_core.mes

In [26]:
chain.invoke({"messages": [HumanMessage(content="Hi My name is benky")]})

AIMessage(content="Hi Benky! It's nice to meet you. 😊 \n\nI'm happy to help with any questions you have. Just ask away!  What can I do for you today? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 32, 'total_tokens': 77, 'completion_time': 0.081818182, 'prompt_time': 0.002370024, 'queue_time': 0.242226434, 'total_time': 0.084188206}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c7881f07-251a-471f-b895-3a7c4f429a04-0', usage_metadata={'input_tokens': 32, 'output_tokens': 45, 'total_tokens': 77})

In [27]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

In [28]:
with_message_history

RunnableWithMessageHistory(bound=RunnableBinding(bound=RunnableBinding(bound=RunnableLambda(_enter_history), kwargs={}, config={'run_name': 'load_history'}, config_factories=[])
| RunnableBinding(bound=RunnableLambda(_call_runnable_sync), kwargs={}, config={'run_name': 'check_sync_or_async'}, config_factories=[]), kwargs={}, config={'run_name': 'RunnableWithMessageHistory'}, config_factories=[]), kwargs={}, config={}, config_factories=[], get_session_history=<function get_session_history at 0x000002A7CFC2D8A0>, history_factory_config=[ConfigurableFieldSpec(id='session_id', annotation=<class 'str'>, name='Session ID', description='Unique identifier for a session.', default='', is_shared=True, dependencies=None)])

In [29]:
config = {"configurable": {"session_id":"chat3"}}

In [30]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Benky")],
    config=config

)

In [31]:
response

AIMessage(content="Hello Benky! It's nice to meet you. \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 33, 'total_tokens': 76, 'completion_time': 0.078181818, 'prompt_time': 0.002364383, 'queue_time': 0.244558577, 'total_time': 0.080546201}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--dff0446a-3dd4-447e-9995-2008e0dc8147-0', usage_metadata={'input_tokens': 33, 'output_tokens': 43, 'total_tokens': 76})

In [32]:
response.content

"Hello Benky! It's nice to meet you. \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😊 \n\n"

In [33]:
# Add More Complexity

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful assistant. Answer all questions to the best of your ability in {language}."),
    MessagesPlaceholder(variable_name="messages")
])

In [34]:
chain = prompt|model

In [35]:
chain

ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annotated[langch

In [36]:
response = chain.invoke({"messages":[HumanMessage(content="Hi My name is Benky")],"language":"Telugu"})

In [37]:
response

AIMessage(content='నమస్తే బెంకి, తేనె నేను మీకు సహాయం చేయగల సుందర సహాయకుడు. ఏదైనా ప్రశ్న ఉంటే అడిగండి! 😊 \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 33, 'total_tokens': 90, 'completion_time': 0.103636364, 'prompt_time': 0.002478824, 'queue_time': 0.241745456, 'total_time': 0.106115188}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--5b1fed59-3973-4ebf-b52f-ee0b3f2338f6-0', usage_metadata={'input_tokens': 33, 'output_tokens': 57, 'total_tokens': 90})

In [38]:
response.content

'నమస్తే బెంకి, తేనె నేను మీకు సహాయం చేయగల సుందర సహాయకుడు. ఏదైనా ప్రశ్న ఉంటే అడిగండి! 😊 \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [39]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

In [41]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke({
    "messages": [HumanMessage(content="Hi I am Benky")],
    "language": "Telugu"
}, config=config)

In [42]:
response.content

'మీకు కల్యాణం Benky! 😊 \n\nనేను మీకు ఎలా సహాయం చేయగలను?  \n\n'

In [43]:
response = with_message_history.invoke({
    "messages": [HumanMessage(content="What is my name?")],
    "language": "Telugu"
}, config=config)

In [44]:
response.content

'మీ పేరు Benky అని తెలుసు.  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [45]:
from langchain_core.messages import SystemMessage, trim_messages

In [46]:
trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)

In [47]:
messages = [
    SystemMessage(content="You're a good assistant"),
    HumanMessage(content="Hi! I'm bob"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="Nice"),
    HumanMessage(content="Whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No problem!"),
    HumanMessage(content="Having fun?"),
    AIMessage(content="Yes!"),
]

In [48]:
trimmer.invoke(messages)

C:\Users\004NQ8744\OneDrive\Roadwork\langChain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\004NQ8744\OneDrive\Roadwork\langChain\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\004NQ8744\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In orde

[SystemMessage(content="You're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})]

In [49]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [50]:
chain = RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer) | prompt | model

In [51]:
chain

RunnableAssign(mapper={
  messages: RunnableLambda(itemgetter('messages'))
            | RunnableLambda(...)
})
| ChatPromptTemplate(input_variables=['language', 'messages'], input_types={'messages': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.An

In [52]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="What Ice Cream do I like?")],
    "language": "English"
})

In [53]:
response.content

"As a large language model, I don't have access to your personal information, including your favorite ice cream flavor.  \n\nWhat's your favorite ice cream?  🍦😊\n"

In [54]:
response = chain.invoke({
    "messages": messages + [HumanMessage(content="What math problem did I ask?")],
    "language": "English"
})

In [55]:
response.content

'You asked "Whats 2 + 2".  😊\n'

In [56]:
# In message history
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)
config = {"configurable": {"session_id":"chat5"}}

In [58]:
response = with_message_history.invoke({
    "messages": messages+[HumanMessage(content="What Math problem did I ask?")],
    "language": "English"
},config=config)

In [59]:
response.content

'You asked: "Whats 2 + 2"  😊  \n\n\n\nIs there anything else I can help you with?\n'